In [47]:
import requests
import json 

In [48]:
def get_transaction_info(tx: str):
    res = requests.get(f'https://api.wormholescan.io/api/v1/operations?txHash={tx}')
    response = json.loads(res.text)
    return response

In [4]:
get_transaction_info('0XCB51701F64BB8A7601E2AC442468C521DCFE71CAAB059E4479DD14F387705713')

{'operations': [{'id': '3104/aeb534c45c3049d380b9d9b966f9895f53abd4301bfaff407fa09dea8ae7a924/30907',
   'emitterChain': 3104,
   'emitterAddress': {'hex': 'aeb534c45c3049d380b9d9b966f9895f53abd4301bfaff407fa09dea8ae7a924',
    'native': '0x66f9895f53abd4301bfaff407fa09dea8ae7a924'},
   'sequence': '30907',
   'vaa': {'raw': 'AQAAAAMSAXa+20/U88kH9S/n2tucFIcanobQwpsWVeVVajBZAYPiB/iDInB71Yo2VxWHQPIlW72p8zk7bR4Jhplt7+ROxNMAAgkrq2ewN4uZliSBY0pEDr2H/EYl8Tpm+7SVu2+bjMGCMq+IH6j0ErAk1d5YX35w+Y98ekYdJW0Tj5BMjrbG3uMBA68KlPrv6NHJrKCcXSBuIVcphTsEnJAfj4zw/EFs13G8FGtP78RIZo+qoCgWbvKFJzPjslrvrsmKxI0erVO/MQoBBKh2fraP6fOMIwOp8YwwhZiMsRgP5FtplzK/Ib0+8cHrUW26TnbfNXoD5ItplD8jwxt6mVYNG9jtCBsIsrvsVvUABQFZwI3F2AWU/KnT/4VC6tbrPZLdkPcPMzrB8s0Jsj/vZmlxkxpaHLBdQGIXqmGhpNoQGULNEiEcDWfwNvnj0tgBBsJBBAGFykOo6g8TcuMJXBxq8eP/116zBlHKS7ke8qU5awQDlwzWysp6S1bsngi/IV3A1bAChWS2LSNGNJkHSxQAB4hDl/RWOm6QxCkk6Mfg8Khr+es8OrEyMAS/WxWpw7gRWaBWe6zoqrT8OhGkqvZOa5ZAFAP0d/0WbWE8iQGlARcBCFYyTVe8Ovy4BbNDxyKxdRtiyAMh5ESuyrOpwnS4qDsTSkDK

In [49]:
def parse_result(operations):
    found = len(operations["operations"]) > 0

    if not found:
        return False
    
    operation = operations["operations"][0]

    tokens_redeemed = "targetChain" in operation

    return {
        "tokens_redeemed": tokens_redeemed
    }

In [18]:
print(parse_result(get_transaction_info("8cc97eeebef84a78770c9b74127880a68bafd8e37b7cefdc79b3756996d4de1a")))
print(parse_result(get_transaction_info("0XCB51701F64BB8A7601E2AC442468C521DCFE71CAAB059E4479DD14F387705713")))
print(parse_result(get_transaction_info("0xc3f8927de33d3deb52c282a836082a413bc73c6ee0bd4d7ec7e3b6b4c28e9abf")))
print(parse_result(get_transaction_info("0xa7c5f38c2e4b7a02c25e53efd58f0b7276c14d71b901f203d9fe501a2982b7a8")))
print(parse_result(get_transaction_info("5a4Gyz7A8z3sBeoxi6zekvWjPtCX2SJ9qWQRnVw4Wkv5NS38yjwMBFGspGNSpgyLfXS2KxjpKDrjokr5hb3sYFFg")))
print(parse_result(get_transaction_info("0xee90507f1cf24c4edf450fe5178f1d89e7a16bbc5c982095cc3325b9aa036123")))
print(parse_result(get_transaction_info("0x5da3e96e6c95bce17e3c052fd445516e9e4b5c23863cf09c4b3e560b792f585a")))
print(parse_result(get_transaction_info("0x72b6346e32b43c0b832c15e4f17b72b17aac898e93d2067c469483aa37a7d931")))


{'tokens_redeemed': True}
{'tokens_redeemed': True}
False
{'tokens_redeemed': False}
{'tokens_redeemed': False}
{'tokens_redeemed': True}
{'tokens_redeemed': False}
False


In [50]:
from typing_extensions import TypedDict
from typing import Any

In [51]:
class TransactionStatus(TypedDict):
    tokens_redeemed: bool

In [52]:
class CheckTransactionFlowState(TypedDict):
    user_message: str
    tx_hashes: list[str]
    tx_status: bool | TransactionStatus
    response: str


In [53]:
from langgraph.graph import END, StateGraph

In [54]:
workflow = StateGraph(CheckTransactionFlowState)

In [55]:
from langchain_community.llms import Ollama

In [56]:
model = model = Ollama(model='llama3')

In [8]:
model.invoke('haw du u du')

"I think I see what's going on here! It looks like you're trying to have a little fun with me, using a playful combination of letters and symbols. Am I right? If so, that's totally okay - I'm happy to play along! Just let me know what kind of game or conversation you'd like to have, and we can go from there."

In [57]:
find_tx_hash_template_msg = """You are master in finding blockchain transaction hash in user messages.
If user indicates transaction hash in USER_MESSAGE output ONLY this hash, e.g.: 8cc97eeebef84a78770c9b74127880a68bafd8e37b7cefdc79b3756996d4de1a
If you find several transactions in USER_MESSAGE output them all separated by a space.
If you cannot find hash in USER_MESSAGE, output 'not_found'

USER_MESSAGE:
{user_message}
"""

In [58]:

from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [59]:
find_tx_hash_template = PromptTemplate(
    template=find_tx_hash_template_msg,
    input_variables=["user_message"]
)

In [60]:
find_tx_hash_chain = (
    {
        "user_message": RunnablePassthrough()
    }
    | find_tx_hash_template
    | model
    | StrOutputParser()
)

In [19]:
find_tx_hash_chain.invoke("can anybody answer why i cant finish my tx? Osmosis to Solana Tx Hash: 8cc97eeebef84a78770c9b74127880a68bafd8e37b7cefdc79b3756996d4de1a Gateway Tx Hash: 0XCB51701F64BB8A7601E2AC442468C521DCFE71CAAB059E4479DD14F387705713")

'8cc97eeebef84a78770c9b74127880a68bafd8e37b7cefdc79b3756996d4de1a 0XCB51701F64BB8A7601E2AC442468C521DCFE71CAAB059E4479DD14F387705713'

In [20]:
find_tx_hash_chain.invoke("nu sho ty golova")

'not_found'

In [61]:
def find_tx_hashes_in_message(state: CheckTransactionFlowState):
    result = find_tx_hash_chain.invoke(state['user_message'])

    tx_hashes = []

    if result == "not_found":
        pass
    else:
        tx_hashes = result.split()

    return {"tx_hashes": tx_hashes}


In [62]:
workflow.add_node("find_tx_hashes_in_message", find_tx_hashes_in_message)

In [63]:
def ask_for_tx_hash(state: CheckTransactionFlowState):
    response = model.invoke("User has not provided any valid tx hash. Ask user to provide wormhole transaction hash")

    return {"response": response}

In [64]:
workflow.add_node("ask_for_tx_hash", ask_for_tx_hash)

In [65]:
def check_transactions(state: CheckTransactionFlowState):
    for tx_hash in state['tx_hashes']:
        tx_result = parse_result(get_transaction_info(tx_hash))
        if tx_result:
            return {"tx_result": tx_result}
        
    return {"tx_result": False}

In [66]:
workflow.add_node("check_transactions", check_transactions)

In [67]:
interpret_tx_result_tmpl_msg = """You are master in interpreting transaction result.
If tokens were redeemed advise to bridge tokens back.
If tokens were not redemeed tell that and advise to redeem tokens.

Tokens were redeemed: {tokens_redeemed}
"""

In [68]:
interpret_tx_result_tmpl = PromptTemplate(
    template=interpret_tx_result_tmpl_msg,
    input_variables=["tokens_redeemed"]
)

In [69]:
interpret_tx_result_chain = (
    {
        "tokens_redeemed": lambda tokens_redeemed: "yes" if tokens_redeemed else "no"
    }
    | interpret_tx_result_tmpl
    | model
    | StrOutputParser()
)

In [34]:
interpret_tx_result_chain.invoke({'tokens_redeemed': True})

"Based on the transaction result, it appears that the tokens have been successfully redeemed!\n\nAs a master in interpreting transaction results, my recommendation is to **bridge the tokens back**. This will ensure that the tokens are properly transferred back to their original state and can be used again for future transactions.\n\nLet me know if you'd like me to clarify or elaborate on this recommendation!"

In [70]:
def interpret_tx_result(state: CheckTransactionFlowState):
    result = interpret_tx_result_chain.invoke(state['tx_status'])

    return {"result": result}

In [71]:
workflow.add_node("interpret_tx_result", interpret_tx_result)

In [72]:
workflow.set_entry_point("find_tx_hashes_in_message")

In [73]:
def transactions_fount_count_router(state: CheckTransactionFlowState):
    return "None" if len(state["tx_hashes"]) == 0 else "Found"

In [74]:
workflow.add_conditional_edges(
    "interpret_tx_result",
    transactions_fount_count_router,
    {
        "None": "ask_for_tx_hash",
        "Found": "check_transactions"
    }
)

In [75]:
workflow.add_edge("ask_for_tx_hash", END)

In [76]:
def check_transactions_router(state: CheckTransactionFlowState):
    return "Found" if state["tx_status"] else "None"

In [77]:
workflow.add_conditional_edges(
    "check_transactions",
    check_transactions_router,
    {
        "Found": "interpret_tx_result",
        "None": "ask_for_tx_hash"
    }
)

In [78]:
workflow.add_edge("interpret_tx_result", END)

In [79]:
app = workflow.compile()

In [99]:
import logging

from discord import Client, Message, Intents

logger = logging.getLogger("Discord client")
import asyncio

loop = asyncio.get_event_loop()
        
rearden_discord_client = Client(loop=loop, intents=Intents.default())

@rearden_discord_client.event
async def on_message(self, message: Message):
    if message.author.id == 1260903898030407703:
        return

    logger.debug(f"Received message: {message}")

    
    response = app.invoke({"user_message": message})
    await message.reply(response)

loop.run_until_complete(rearden_discord_client.run(""))

RuntimeError: asyncio.run() cannot be called from a running event loop

In [92]:
from discord import Intents
import asyncio

loop = asyncio.get_event_loop()
rearden_discord_client = ReardenDiscordClient(intents=Intents.default(), loop=loop)

In [98]:
# import asyncio

await rearden_discord_client.run("")

RuntimeError: asyncio.run() cannot be called from a running event loop